In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, DepthwiseConv2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, BatchNormalization
from keras.layers import Add, Multiply, Reshape
from keras.utils.vis_utils import plot_model

In [ ]:
# activation function
def relu6(x):
    return K.relu(x, max_value=6.0)

def hard_swish(x):
    return x * K.relu(x + 3.0, max_value=6.0) / 6.0

In [ ]:
def Act(x, nl):
    """
    choice activation function
    
    # params
    x: Tensor, input tensor of conv layer
    nl: String, nonlinear  activation function type
    """
    if nl == "HS":
        x = Activation(hard_swish)(x)
    if nl == "RE":
        x = Activation(relu6)(x)
    
    return x

In [ ]:
def Conv(inputs, filters, kernel, strides, nl):
    """
    Convolution Block defines 2D convolution operation with BN and activation
    
    # params
    inputs: Tensor, input tensor of convolution layer
    filters: Integer, dimension of output space
    kernel: Integer or tuple/list of 2 integers
            specifying width and height of 2D convolution window
    strides: Integer or tuple/list of 2 integers
             specifying strides of convolution width and height
    nl: String, nonlinear activation function type
    
    # return
    output tensor
    """
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    
    x = Conv2D(filters, kernel, padding="same", strides=strides, use_bias=False)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Act(x, nl)
        
    return x

In [ ]:
def Squeeze(inputs):
    """
    Squeeze and Excitation define squeeze structure
    
    # params
    inputs: Tensor, input tensor of conv layer
    """
    channels = int(inputs.shape[-1])
    
    x = GlobalAveragePooling2D()(inputs)
    x = Dense(channels, activation="relu")(x)
    x = Dense(channels, activation="hard_sigmoid")(x)
    x = Reshape((1, 1, channels))(x)
    x = Multiply()([inputs, x])
    
    return x

In [ ]:
def bottleneck(inputs, filters, kernel, e, s, squeeze, nl, alpha=1.0):
    """
    Bottleneck define basic bottleneck structure
    
    # params
    inputs: Tensor, input tensor of conv layer
    filters: Integer, dimension of output space
    kernel: Integer of tuple/list of 2 integers
            specifying width and height of 2D conv window
    e: Integer, expansion factor
       t is always applied to input size
    s: Integer of tuple/list of 2 integers
       specifying strides of conv width and height
    squeeze: Boolean, whether to use squeeze
    nl: String, nonlinear activation function type
    alpha: Integer, width multiplier
           If alpha < 1.0, proportionally decreases the number of filters in each layer.
           If alpha > 1.0, proportionally increases the number of filters in each layer.
           If alpha = 1.0, default number of filters from the paper are used at each layer.
    
    # return 
    output tensor
    """
    
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    input_shape = tf.keras.backend.int_shape(inputs)
    
    tchannel = int(e)
    cchannel = int(alpha * filters)
    
    r = s == 1 and input_shape[3] == filters
    
    x = Conv(inputs, tchannel, (1, 1), (1, 1), nl)
    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding="same")(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Act(x, nl)
    
    if squeeze:
        x = Squeeze(x)
    
    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding="same", use_bias=False)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    
    if r:
        x = Add()([x, inputs])
    
    return x

In [ ]:
def build(shape=(224, 224, 3), n_class=1000, alpha=1.0, include_top=True, pooling=None, plot=False):
    """
    build MobilNetV3 Small
    
    # params
    shape
    """
    inputs = Input(shape=shape)
    
    x = Conv(inputs, 16, kernel=3, strides=2, nl="HS")
    
    x = bottleneck(x, 16, kernel=3, e=16, s=2, squeeze=True, nl="RE")
    x = bottleneck(x, 24, kernel=3, e=72, s=2, squeeze=False, nl="RE")
    x = bottleneck(x, 24, kernel=3, e=88, s=1, squeeze=False, nl="RE")
    x = bottleneck(x, 40, kernel=5, e=96, s=2, squeeze=True, nl="HS")
    x = bottleneck(x, 40, kernel=5, e=240, s=1, squeeze=True, nl="HS")
    x = bottleneck(x, 40, kernel=5, e=240, s=1, squeeze=True, nl="HS")
    x = bottleneck(x, 48, kernel=5, e=120, s=1, squeeze=True, nl="HS")
    x = bottleneck(x, 48, kernel=5, e=144, s=1, squeeze=True, nl="HS")
    x = bottleneck(x, 96, kernel=5, e=288, s=2, squeeze=True, nl="HS")
    x = bottleneck(x, 96, kernel=5, e=576, s=1, squeeze=True, nl="HS")
    x = bottleneck(x, 96, kernel=5, e=576, s=1, squeeze=True, nl="HS")
    
    x = Conv(x, 576, kernel=1, strides=1, nl="HS")
    # feature extraction
    #-------------------------------------------------------------------#
    # pooling - none(default), avg(global average pooling), max(global max pooling)
    if pooling == 'avg':
        x = GlobalAveragePooling2D()(x)
        x = Reshape((1, 1, 576))(x)

        x = Conv2D(1280, kernel_size=1, padding="same")(x)
        x = Act(x, "HS")
        
    if pooling == 'max':
        x = GlobalMaxPooling2D()(x)
        x = Reshape((1, 1, 576))(x)

        x = Conv2D(1280, kernel_size=1, padding="same")(x)
        x = Act(x, "HS")
        
    # if include_top is False, can construct custom layers
    if include_top:
        x = Conv2D(n_class, kernel_size=1, padding="same", activation="softmax")(x)
        x = Reshape((n_class,))(x)
        
    model = Model(inputs, x)
    
    if plot:
        plot_model(model, to_file="MobileNetV3_small.png", show_shapes=True)
    
    return model

In [ ]:
def train():
    model = build(plot=True, include_top=False)

In [ ]:
model = build(plot=True, include_top=False)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             